In [4]:
import requests
import pandas as pd
import json
from datetime import date, timedelta
import logging


In [5]:
def flatten_dict(dd, separator='_', prefix=''):
    return {f"{prefix}{separator}{k}" if prefix else k: v
            for kk, vv in dd.items()
            for k, v in (flatten_dict(vv, separator, kk).items() if isinstance(vv, dict) else {kk: vv}.items())}

In [6]:
def choose_flight(day1, month1, year1, day2, month2, year2, orig, dest):
    url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"

    start_date = date(year1, month1, day1)
    end_date = date(year2, month2, day2)
    delta = end_date - start_date
    date_range = [start_date + timedelta(days=i) for i in range(delta.days + 1)]
    
    all_quotes = []

    for i in date_range:
        fixed_date = {
            "year": i.year,
            "month": i.month,
            "day": i.day
        }

        payload = {
            "query": {
                "market": "UK",
                "locale": "en-GB",
                "currency": "EUR",
                "queryLegs": [
                    {
                        "originPlace": {
                            "queryPlace": {
                                "iata": orig
                            }
                        },
                        "destinationPlace": {
                            "queryPlace": {
                                "iata": dest
                            }
                        },
                        "fixedDate": fixed_date
                    }
                ]
            }
        }
        headers = {
            "x-api-key": "prtl6749387986743898559646983194"
        }

        try:
            response = requests.post(url, json=payload, headers=headers)
            response.raise_for_status()
        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP error occurred: {err}")
            continue
        except Exception as err:
            logging.error(f"An error occurred: {err}")
            continue

        try:
            output = response.json()
            results = output['content']['results']
            quotes = list(results['quotes'].values())
            flattened_results = [flatten_dict(quote) for quote in quotes]
            all_quotes.append(pd.DataFrame.from_records(flattened_results))
        except KeyError as err:
            logging.error(f"Expected key missing in response: {err}")
            continue
        except Exception as err:
            logging.error(f"An error occurred while parsing response: {err}")
            continue

    quote_df = pd.concat(all_quotes)
    return quote_df

In [7]:
quote_df = choose_flight(1, 8, 2023, 4, 8, 2023, 'LIS', 'LTN')
quote_df

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,31,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,8,1,0,...,,,0,0,0,0,0,0,0,
0,44,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,8,2,0,...,,,0,0,0,0,0,0,0,
0,56,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,8,3,0,...,,,0,0,0,0,0,0,0,
0,54,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,8,4,0,...,,,0,0,0,0,0,0,0,
